In [119]:
import pandas as pd
import numpy as np

In [120]:
input_file_path = r'C:\Users\Kobiz\Documents\Term_2\Computer_Vision\Project\LicensePlates\output.xlsx'
output_summary_file_path = r'C:\Users\Kobiz\Documents\Term_2\Computer_Vision\Project\LicensePlates\summary_data.xlsx'
data = pd.read_excel(input_file_path)

In [121]:
acc_raw = data['Accuracy']
text_raw = data['Text']
pic_path_raw = data['File path']

In [122]:
first = acc_raw > 0.5
first_filtered = data[first]

In [123]:
def thaiLanguage(text):
    return any(ord(char) >= 3584 and ord(char) <= 3711 for char in text)

thaiVowels = set(['ะ', 'า', 'ิ', 'ี', 'ื', 'ึ', 'เ', 'โ', '๊', 'ั', '้', '็', '๊', 'ํ', '๋', '์', 'ุ', 'ู', 'ใ', 'ๆ', 'ไ', 'ฯ', 'ำ'])
def thaiVowel(text):
    return any(vowel in text for vowel in thaiVowels)

def symbols(text):
    return any(not char.isalnum() for char in text)

def num(text):
    return any(char.isdigit() for char in text)
thaiNum = set(['๑', '๒', '๓', '๔', '๕', '๖', '๗', '๘', '๙'])
def thnum(text):
    return any(numeral in text for numeral in thaiNum)

In [124]:
def numbftex(text):
    count = 0
    for char in text:
        if char.isdigit():
            count += 1
        elif ord(char) >= 3584 and ord(char) <= 3711:  # Check for Thai language characters
            if(count <= 1):
                return True
            else:
                return False  # Return True if not more than one number before Thai language

In [125]:
def cNumAf(text):
    count = 0
    fTH = False

    for char in reversed(text):
        if ord(char) >= 3584 and ord(char) <= 3711:  # Check for Thai language characters
            fTH = True
            break
        elif char.isdigit():
            count += 1

    return fTH and 1 <= count <= 4

In [126]:
def thai2ch(text):
    THcount = sum(1 for char in text if 3584 <= ord(char) <= 3711)  # Count Thai language characters
    if THcount == 2:
        return True
    else:
        return False

In [127]:
def groupping(text):
    numbers = ""
    found_thai_language = False

    for char in reversed(text):
        if ord(char) >= 3584 and ord(char) <= 3711:  # Check for Thai language characters
            found_thai_language = True
            break
        elif char.isdigit():
            numbers = char + numbers

    return numbers if found_thai_language else None


In [128]:
def firstTH(text):
    for char in text:
        if 3584 <= ord(char) <= 3711:  # Check for Thai language characters
            return char
    return None

In [129]:
def mcth(group):
    counts = group['FirstThaiCharacter'].value_counts()
    most_common_char = counts.idxmax()
    group['First Thai Character'] = most_common_char
    return group

In [130]:
def cartype(char):
    sedan_chars = ['ก', 'ข', 'จ', 'ฉ', 'ช', 'ฌ', 'ญ', 'ฎ', 'ฐ', 'ธ', 'พ', 'ภ', 'ว', 'ศ', 'ษ', 'ส']
    microbus_chars = ['น', 'ฬ', 'อ', 'ฮ']
    van_pickup_chars = ['ฒ', 'ณ', 'ต', 'ถ', 'บ', 'ผ', 'ย', 'ร', 'ล']
    motortricycle_char = 'ศ'
    interprovincial_taxi_char = 'ว'
    urban_taxi_chars = ['ท', 'ม']
    Car_For_Hire = 'ฎ'

    if char in sedan_chars:
        return 'Sedan'
    elif char in microbus_chars:
        return 'Microbus & Passenger Van'
    elif char in van_pickup_chars:
        return 'Van & Pick Up'
    elif char == motortricycle_char:
        return 'Motortricycle'
    elif char == interprovincial_taxi_char:
        return 'Interprovincial Taxi'
    elif char in urban_taxi_chars:
        return 'Urban Taxi'
    elif char in Car_For_Hire:
        return 'Car for hire'
    else:
        return 'Other'
    

In [131]:
def sort_group(group):
    return group.sort_values(by='Accuracy', ascending=False)

In [132]:
def MaxAcc(group):
    return group.head(1)

In [133]:
from click import group

second = first_filtered['Text'].apply(thaiLanguage)
second_filtered = first_filtered[second]

third = ~second_filtered['Text'].apply(thaiVowel)
third_filtered = second_filtered[third]

forth = ~third_filtered['Text'].apply(symbols)
forth_filtered = third_filtered[forth]

fifth = forth_filtered['Text'].apply(num)
fifth_filtered = forth_filtered[fifth]

sixth = ~fifth_filtered['Text'].apply(thnum)
sixth_filtered = fifth_filtered[sixth]

seventh = sixth_filtered['Text'].apply(numbftex)
seventh_filtered = sixth_filtered[seventh]

eight = seventh_filtered['Text'].apply(cNumAf)
eight_filtered = seventh_filtered[eight]

nineth = eight_filtered['Text'].apply(thai2ch)
nineth_filtered = eight_filtered[nineth]
nineth_filtered['Group'] = nineth_filtered['Text'].apply(groupping)
nineth_filtered['FirstThaiCharacter'] = nineth_filtered['Text'].apply(firstTH)

group_data = nineth_filtered.groupby('FirstThaiCharacter').apply(mcth)
group_data['Car Type'] = group_data['First Thai Character'].apply(cartype)

newFile_grouped = group_data.groupby(['Group', 'First Thai Character', 'Car Type']).first().reset_index()

sorted_new = newFile_grouped.groupby('Group').apply(sort_group)

maxAcc = newFile_grouped.groupby('Group').apply(MaxAcc)

Reorder = ['File path', 'Text', 'Group', 'First Thai Character', 'Car Type', 'Accuracy']
final = maxAcc[Reorder]



C:\Users\Kobiz\AppData\Local\Temp\ipykernel_1304\2000843020.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nineth_filtered['Group'] = nineth_filtered['Text'].apply(groupping)
C:\Users\Kobiz\AppData\Local\Temp\ipykernel_1304\2000843020.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nineth_filtered['FirstThaiCharacter'] = nineth_filtered['Text'].apply(firstTH)


In [134]:
final.to_excel(output_summary_file_path, index=False)
print(f"Summary data saved to {output_summary_file_path}")

Summary data saved to C:\Users\Kobiz\Documents\Term_2\Computer_Vision\Project\LicensePlates\summary_data.xlsx
